In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, accuracy_score
from scipy import stats
from Bio import SeqIO
from adopt.utils import fasta_to_df, df_to_fasta

/home/ubuntu/python3-venv/lib/python3.8/site-packages/tqdm-4.64.0-py3.8.egg/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Functions

In [34]:
def no_go_ixs(zscores):
    no_go_ixs = [i for i, x in enumerate(zscores) if x == 999.0]
    return no_go_ixs

def take_go_pos(scores, no_go_pos):
    go_pos = [scores[i] for i in range(len(scores)) if i not in no_go_pos]
    return go_pos

def put_nans(x):
    new_scores = [i if i != 999.0 else np.nan for i in x]
    return new_scores

def to_bin(x):
    bin_scores = []
    for score in x:
        if score < 8:
            new_score = 1
        elif score >=8:
            new_score = 0
        else:
            new_score = score
        bin_scores.append(new_score)
    return bin_scores

def get_dis_regions(z_score_bin, dis_reg_len):
    zones = {"ones": [], "zeros": [], 'ones_region': []}
    reg = 0
    for n, i in enumerate(z_score_bin):
        pos_reg_diff = n-dis_reg_len+1
        pos_reg_sum = n+dis_reg_len
        #if n>0:
        if pos_reg_diff>=0:
            #if z_score_bin[n]+z_score_bin[n-1])==2:
            if (sum(z_score_bin[pos_reg_diff:n+1]))==dis_reg_len:
                zones['ones'].append(n)
                zones['ones_region'].append(reg)        
            else:
                zones['zeros'].append(n)
                reg+=1
        else:
            #if (z_score_bin[n]+z_score_bin[n+1])==2:
            if (sum(z_score_bin[n:pos_reg_sum]))==dis_reg_len:
                zones['ones'].append(n)
                zones['ones_region'].append(reg)   
            else:
                zones['zeros'].append(n)
    return zones

def get_disorder_reg_index(bin_true_score, dis_reg_len):
    dis_regions = get_dis_regions(bin_true_score, dis_reg_len)['ones_region']
    dis_regions_ixs = get_dis_regions(bin_true_score, dis_reg_len)['ones']

    regions = list(set(dis_regions))
    dis_reg_ix_list = []

    for n, reg in enumerate(regions):
        region_ix = [i for i, j in enumerate(dis_regions) if j == reg]
        dis_reg_ix = [dis_regions_ixs[n] for n in region_ix]
        dis_reg_ix_list.append(dis_reg_ix)
    return dis_reg_ix_list

def compute_accuracy_per_region(true_score, pred_score, bin_true_score, bin_pred_score, dis_ixs):
    #accuracy_prot = []
    #corr_prot = []
    true_z_score_dis_l = []
    pred_z_score_dis_l = []
    dis_reg_true_clss = []
    dis_reg_pred_clss = []
    for reg_ixs in dis_ixs:
        true_z_score_class_dis_reg = [bin_true_score[i] for i in reg_ixs]
        pred_z_score_class_dis_reg = [bin_pred_score[i] for i in reg_ixs]
        true_z_score_dis_reg = [true_score[i] for i in reg_ixs]
        pred_z_score_dis_reg = [pred_score[i] for i in reg_ixs]
        #acc = accuracy_score(true_z_score_class_dis_reg, pred_z_score_class_dis_reg)
        #prs = stats.pearsonr(true_z_score_dis_reg, pred_z_score_dis_reg)
        #accuracy_prot.append(acc)
        #corr_prot.append(prs)
        dis_reg_true_clss.extend(true_z_score_class_dis_reg)
        dis_reg_pred_clss.extend(pred_z_score_class_dis_reg)
        true_z_score_dis_l.extend(true_z_score_dis_reg)
        pred_z_score_dis_l.extend(pred_z_score_dis_reg)
    return true_z_score_dis_l, pred_z_score_dis_l, dis_reg_true_clss, dis_reg_pred_clss

def extract_disordered_regions_bin(bin_true_score, bin_pred_score, dis_ixs, perc_dis):
    dis_reg_true_clss = []
    dis_reg_pred_clss = []
    for reg_ixs in dis_ixs:
        true_z_score_class_dis_reg = [bin_true_score[i] for i in reg_ixs]
        pred_z_score_class_dis_reg = [bin_pred_score[i] for i in reg_ixs]
        n_true_dis_res = sum(true_z_score_class_dis_reg)
        n_pred_dis_res = sum(pred_z_score_class_dis_reg)
        fraction_of_true_disorder = (np.rint(len(true_z_score_class_dis_reg)*perc_dis)).astype(int)
        fraction_of_pred_disorder = (np.rint(len(pred_z_score_class_dis_reg)*perc_dis)).astype(int)
        
        if n_true_dis_res >= fraction_of_true_disorder:
            dis_reg_true_clss.append(1)
        else: 
            dis_reg_true_clss.append(0)
            
        if n_pred_dis_res >= fraction_of_pred_disorder:
            dis_reg_pred_clss.append(1)
        else: 
            dis_reg_pred_clss.append(0)
        
    return dis_reg_true_clss, dis_reg_pred_clss

def extract_disordered_regions(bin_true_score, bin_pred_score, dis_ixs):
    dis_reg_true_frac = []
    dis_reg_pred_frac = []
    for reg_ixs in dis_ixs:
        true_z_score_class_dis_reg = [bin_true_score[i] for i in reg_ixs]
        pred_z_score_class_dis_reg = [bin_pred_score[i] for i in reg_ixs]
        n_true_dis_res = sum(true_z_score_class_dis_reg)
        n_pred_dis_res = sum(pred_z_score_class_dis_reg)
        fraction_of_true_disorder = float(n_true_dis_res/len(true_z_score_class_dis_reg))
        fraction_of_pred_disorder = float(n_pred_dis_res/len(pred_z_score_class_dis_reg))
        #print(fraction_of_pred_disorder)
        
        dis_reg_true_frac.append(fraction_of_true_disorder)
        dis_reg_pred_frac.append(fraction_of_pred_disorder)       
        
    return dis_reg_true_frac, dis_reg_pred_frac

def trim_seq(x, trim_fac):
    if isinstance(x, list):
        x_trim = x[:-trim_fac]
    else:
        x_trim = list(x)
        x_trim = x_trim[:-trim_fac]
        x_trim = ''.join(x_trim)
    return x_trim

# Parameters

In [19]:
dis_reg_len = 3
trim_fac = 10

# Trim sequences

In [4]:
df_117_orig = fasta_to_df("../datasets/chezod_117_all.fasta")

In [5]:
df_117_trim = df_117_orig.copy()
df_117_trim['sequence'] = df_117_trim.sequence.apply(trim_seq, trim_fac=trim_fac)

In [6]:
df_to_fasta(df_117_trim, f"../datasets/chezod_117_all_trim_{trim_fac}.fasta")

# Import data

### Test

In [25]:
chezod_117_pred_path = '../results/chezod_117_predicted_z_scores.json'
chezod_117_true_path = '../datasets/117_dataset_raw.json'

df_117_pred = pd.read_json(chezod_117_pred_path, orient='records')
df_117_true = pd.read_json(chezod_117_true_path, orient='records')

### Trim test

In [36]:
chezod_117_pred_path = f'../results/chezod_117_predicted_z_scores_trim_{trim_fac}.json'
chezod_117_true_path = '../datasets/117_dataset_raw.json'

df_117_pred = pd.read_json(chezod_117_pred_path, orient='records')
df_117_true = pd.read_json(chezod_117_true_path, orient='records')
df_117_true['sequence'] = df_117_true.sequence.apply(trim_seq, trim_fac=trim_fac)
df_117_true['zscore'] = df_117_true.zscore.apply(trim_seq, trim_fac=trim_fac)

# Preprocessing

In [41]:
df_117 = pd.concat([df_117_true, df_117_pred[["z_scores"]]], axis=1)
df_117.rename(columns={"zscore": "zscore_true", "z_scores": "zscore_pred"}, inplace=True)

In [42]:
df_117["no_go_ixs"] = df_117.zscore_true.apply(no_go_ixs)
df_117["zscore_true_nan"] = df_117.zscore_true.apply(put_nans)
df_117["go_zscore_true"] = df_117.apply(lambda x: take_go_pos(x['zscore_true'], x['no_go_ixs']), axis=1)
df_117["go_zscore_pred"] = df_117.apply(lambda x: take_go_pos(x['zscore_pred'], x['no_go_ixs']), axis=1)
df_117["zscore_true_bin"] = df_117.zscore_true_nan.apply(to_bin)
df_117["zscore_pred_bin"] = df_117.zscore_pred.apply(to_bin)

In [43]:
zscores_true = np.concatenate(df_117.zscore_true_nan.values)
zscores_pred = np.concatenate(df_117.zscore_pred.values)

In [44]:
go_zscores_true = np.concatenate(df_117.go_zscore_true.values)
go_zscores_pred = np.concatenate(df_117.go_zscore_pred.values)

# Statistical analysis

In [45]:
mean_absolute_error(go_zscores_true, go_zscores_pred)

3.360727014360871

In [46]:
stats.spearmanr(zscores_true, zscores_pred, nan_policy='omit')

SpearmanrResult(correlation=0.6945910326085772, pvalue=0.0)

In [47]:
stats.pearsonr(go_zscores_true, go_zscores_pred)

(0.7250096213283791, 0.0)

In [48]:
bin_true_scores = df_117.zscore_true_bin.values
bin_pred_scores = df_117.zscore_pred_bin.values

true_scores = df_117.zscore_true.values
pred_scores = df_117.zscore_pred.values

# Z score corr in disordered regions

In [49]:
dis_reg_true_zscore = []
dis_reg_pred_zscore = []
dis_reg_true_class = []
dis_reg_pred_class = []


for i in range(len(true_scores)):
    bin_true_score = bin_true_scores[i] 
    bin_pred_score = bin_pred_scores[i] 
    true_score = true_scores[i] 
    pred_score = pred_scores[i] 
    dis_ixs = get_disorder_reg_index(bin_true_score, dis_reg_len=dis_reg_len)
    true_z_score_dis_reg, pred_z_score_dis_reg, dis_reg_true_clss, dis_reg_pred_clss = compute_accuracy_per_region(true_score,
                                                                                                                   pred_score, 
                                                                                                                   bin_true_score, 
                                                                                                                   bin_pred_score, 
                                                                                                                   dis_ixs)
    dis_reg_true_zscore.extend(true_z_score_dis_reg)
    dis_reg_pred_zscore.extend(pred_z_score_dis_reg)
    dis_reg_true_class.extend(dis_reg_true_clss)
    dis_reg_pred_class.extend(dis_reg_pred_clss)

In [50]:
stats.spearmanr(dis_reg_true_zscore, dis_reg_pred_zscore)

SpearmanrResult(correlation=0.3075198199734613, pvalue=1.126664906230272e-163)

In [51]:
stats.pearsonr(dis_reg_true_zscore, dis_reg_pred_zscore)

(0.3345466357133406, 3.7222839454334604e-195)

# Accuracy in disordered regions

### Per residue

In [52]:
accuracy_score(dis_reg_true_class, dis_reg_pred_class)

0.8263193052772211

### Per region

In [53]:
dis_reg_true = []
dis_reg_pred = []
dis_reg_true_prob = []
dis_reg_pred_prob = []


for i in range(len(true_scores)):
    bin_true_score = bin_true_scores[i] 
    bin_pred_score = bin_pred_scores[i] 
    dis_ixs = get_disorder_reg_index(bin_true_score, dis_reg_len=dis_reg_len)
    dis_reg_true_clss, dis_reg_pred_clss = extract_disordered_regions_bin(bin_true_score, 
                                                                          bin_pred_score, 
                                                                          dis_ixs,
                                                                          perc_dis=0.7)
    dis_reg_true_frac, dis_reg_pred_frac = extract_disordered_regions(bin_true_score, 
                                                                      bin_pred_score, 
                                                                      dis_ixs)
    
    dis_reg_true.extend(dis_reg_true_clss)
    dis_reg_pred.extend(dis_reg_pred_clss)
    dis_reg_true_prob.extend(dis_reg_true_frac)
    dis_reg_pred_prob.extend(dis_reg_pred_frac)

In [54]:
accuracy_score(dis_reg_true, dis_reg_pred)

0.6488095238095238